In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
from astropy.table import Table, Column
from tqdm import tqdm
import math
from astropy.io import fits
import astropy.units as u

from scipy.interpolate import RegularGridInterpolator
#from dust_extinction.parameter_averages import F99
#from dust_extinction.parameter_averages import G23
from scipy.signal import savgol_filter

from scipy.optimize import curve_fit
import matplotlib.cm as cm

from scipy.stats import gmean, hmean

In [2]:
s = Table.read("wtts_veiling.fits")


lower_temps, upper_temps, median_veilings, lower_percentile_veilings, upper_percentile_veilings = [], [], [], [], []

indices = np.where(s['pred_logteff_1'] < np.log10(2300))[0]
if len(indices) > 0:
    veiling_teff = np.array(s['veiling_arr'][indices])
    median_veiling = np.nanquantile(veiling_teff, 0.5, axis=0)
    lower_percentile_veiling = np.nanquantile(veiling_teff, 0.16, axis=0)
    upper_percentile_veiling = np.nanquantile(veiling_teff, 0.84, axis=0)
else:
    median_veiling = np.full(70, np.nan)
    lower_percentile_veiling = np.full(70, np.nan)
    upper_percentile_veiling = np.full(70, np.nan)

lower_temps.append(0)
upper_temps.append(2300)
median_veilings.append(median_veiling)
lower_percentile_veilings.append(lower_percentile_veiling)
upper_percentile_veilings.append(upper_percentile_veiling)

for lower_temp in range(2300, 11000, 10):
    upper_temp = lower_temp + 10
    indices = np.where((s['pred_logteff_1'] >= np.log10(lower_temp)) & (s['pred_logteff_1'] < np.log10(upper_temp)))[0]
    if len(indices) > 0:
        veiling_teff = np.array(s['veiling_arr'][indices])
        median_veiling = np.nanquantile(veiling_teff, 0.5, axis=0)
        lower_percentile_veiling = np.nanquantile(veiling_teff, 0.16, axis=0)
        upper_percentile_veiling = np.nanquantile(veiling_teff, 0.84, axis=0)
    else:
        median_veiling = np.full(70, np.nan)
        lower_percentile_veiling = np.full(70, np.nan)
        upper_percentile_veiling = np.full(70, np.nan)

    lower_temps.append(lower_temp)
    upper_temps.append(upper_temp)
    median_veilings.append(median_veiling)
    lower_percentile_veilings.append(lower_percentile_veiling)
    upper_percentile_veilings.append(upper_percentile_veiling)

indices = np.where(s['pred_logteff_1'] >= np.log10(11000))[0]
if len(indices) > 0:
    veiling_teff = np.array(s['veiling_arr'][indices])
    median_veiling = np.nanquantile(veiling_teff, 0.5, axis=0)
    lower_percentile_veiling = np.nanquantile(veiling_teff, 0.16, axis=0)
    upper_percentile_veiling = np.nanquantile(veiling_teff, 0.84, axis=0)
else:
    median_veiling = np.full(70, np.nan)
    lower_percentile_veiling = np.full(70, np.nan)
    upper_percentile_veiling = np.full(70, np.nan)

lower_temps.append(11000)
upper_temps.append(np.inf)
median_veilings.append(median_veiling)
lower_percentile_veilings.append(lower_percentile_veiling)
upper_percentile_veilings.append(upper_percentile_veiling)

t = Table()
t['Lower_Temp'] = Column(lower_temps, unit=u.K, description='Lower bound of temperature range')
t['Upper_Temp'] = Column(upper_temps, unit=u.K, description='Upper bound of temperature range')
t['Median_Veiling'] = median_veilings
t['Lower_Uncertainty'] = np.array(median_veilings) - np.array(lower_percentile_veilings)
t['Upper_Uncertainty'] = np.array(median_veilings) - np.array(upper_percentile_veilings)

arr = np.linspace(3566.9744, 10318.104, 70)
t["Wavelength"] = Column(length=len(t),dtype=float, shape=(70,))+np.nan
t["Wavelength"] = arr

t.write('binned_wtts_veiling.fits', overwrite=True)


/home/serat/miniconda3/envs/stenv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


In [8]:
for i in range(len(t)):
    print(t['Upper_Temp'][i], t['Median_Veiling'][i])

2300.0 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
2310.0 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
2320.0 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
2330.0 [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan

In [44]:
t = Table.read('binned_wtts_veiling.fits')

corrected_veiling_list = []

t_ctts = Table.read('ctts_veiling.fits')




veiling = t_ctts['veiling_arr']
wtts_temp = t['Lower_Temp']
wtts_veiling = t['Median_Veiling']


for i in range(len(t)):
    temp = 10**t_ctts['pred_logteff_1'][i]
    temp_range = (temp//10)*10

    a = np.where(wtts_temp == temp_range)[0]
    wtts_veiling_matched = wtts_veiling[a[0]]
    
    corrected_veiling = t_ctts['veiling_arr'][i] - wtts_veiling_matched 
    
    if len(corrected_veiling) > 1:
        t_ctts['veiling_arr'][i] = corrected_veiling

t_ctts.write('teff_corrected_veiling.fits', overwrite=True)


In [42]:
t = t_ctts
teff = 10**t['pred_logteff_1']
eqw = t['Halpha_eqw_1']

# Define the bin edges
bins = np.arange(-110, 101, 10)

# Define the temperature ranges
temperature_ranges = [(3000, 3500), (3500, 4000), (4000, 4500), 
                      (4500, 5000), (5000, 5500), (5500, 6000)]


lower_temp_list = []
upper_temp_list = []
eqw_lower_lim_list = []
eqw_upper_lim_list = []
median_veiling_array_list = []

# Looping over each temperature range
for lower_temp, upper_temp in temperature_ranges:
    indices = np.where((teff >= lower_temp) & (teff < upper_temp))[0]
    if len(indices) > 0:
        eqw_teff = np.array(eqw[indices])
        veiling_teff = np.array(t['veiling_arr'][indices])

        # Iterate over bins to calculate median veiling for each bin
        for i in range(len(bins) - 1):
            bin_indices = np.where((eqw_teff >= bins[i]) & (eqw_teff < bins[i + 1]))[0]
            if len(bin_indices) > 0:
                # Calculate the median veiling for this bin (it will be an array)
                median_veiling = np.nanmedian(veiling_teff[bin_indices], axis=0)

                # Store the entire median veiling array if not all NaN
                if not np.isnan(median_veiling).all():
                    lower_temp_list.append(lower_temp)
                    upper_temp_list.append(upper_temp)
                    eqw_lower_lim_list.append(bins[i])
                    eqw_upper_lim_list.append(bins[i + 1])
                    median_veiling_array_list.append(median_veiling)


t = Table([lower_temp_list, upper_temp_list, eqw_lower_lim_list, eqw_upper_lim_list, median_veiling_array_list], 
          names=('lower_temp', 'upper_temp', 'eqw_lower_lim', 'eqw_upper_lim', 'median_veiling'))


t.write('binned_veiling_eqw_array.fits', format='fits', overwrite=True)


/tmp/ipykernel_5057/843859771.py:31: RuntimeWarning: All-NaN slice encountered
  median_veiling = np.nanmedian(veiling_teff[bin_indices], axis=0)


In [43]:
from astropy.table import Table, Column
import numpy as np

# Read the data from the FITS file
t = t_ctts
teff = 10**t['pred_logteff_1']
age = t['age']

# Define the bin edges
bins = np.arange(5.6, 7.9, 0.1)

# Define the temperature ranges
temperature_ranges = [(3000, 3500), (3500, 4000), (4000, 4500), 
                      (4500, 5000), (5000, 5500), (5500, 6000)]

# Initialize lists to store the data for the FITS table
lower_temp_list = []
upper_temp_list = []
age_lower_lim_list = []
age_upper_lim_list = []
median_veiling_array_list = []

# Loop over each temperature range
for lower_temp, upper_temp in temperature_ranges:
    indices = np.where((teff >= lower_temp) & (teff < upper_temp))[0]
    if len(indices) > 0:
        age_teff = np.array(age[indices])
        veiling_teff = np.array(t['veiling_arr'][indices])

        # Iterate over bins to calculate median veiling for each bin
        for i in range(len(bins) - 1):
            bin_indices = np.where((age_teff >= bins[i]) & (age_teff < bins[i + 1]))[0]
            if len(bin_indices) > 0:
                # Calculate the median veiling for this bin (it will be an array)
                median_veiling = np.nanmedian(veiling_teff[bin_indices], axis=0)

                # Store the entire median veiling array if not all NaN
                if not np.isnan(median_veiling).all():
                    lower_temp_list.append(lower_temp)
                    upper_temp_list.append(upper_temp)
                    age_lower_lim_list.append(bins[i])
                    age_upper_lim_list.append(bins[i + 1])
                    median_veiling_array_list.append(median_veiling)

# Create the FITS table
t = Table([lower_temp_list, upper_temp_list, age_lower_lim_list, age_upper_lim_list, median_veiling_array_list], 
          names=('lower_temp', 'upper_temp', 'age_lower_lim', 'age_upper_lim', 'median_veiling'))

#t
t.write('binned_veiling_age_array.fits', format='fits', overwrite=True)


/tmp/ipykernel_5057/686443854.py:35: RuntimeWarning: All-NaN slice encountered
  median_veiling = np.nanmedian(veiling_teff[bin_indices], axis=0)


In [47]:
s = Table.read("teff_corrected_veiling.fits")
#wl = np.linspace(3566.9744, 10318.104, 70)
#a = np.where((s['pred_logteff'] < np.log10(4000)) & (s['pred_logteff'] > np.log10(3000)))[0]
#print(len(a))
#veiling_teff = np.array(s['veiling_arr'][a])
#median_veiling = np.nanmedian(veiling_teff, axis=0)
#print(median_veiling)


lower_temps, upper_temps, median_veilings, lower_percentile_veilings, upper_percentile_veilings = [], [], [], [], []

indices = np.where(s['pred_logteff_1'] < np.log10(2300))[0]
if len(indices) > 0:
    veiling_teff = np.array(s['veiling_arr'][indices])
    median_veiling = np.nanquantile(veiling_teff, 0.5, axis=0)
    lower_percentile_veiling = np.nanquantile(veiling_teff, 0.16, axis=0)
    upper_percentile_veiling = np.nanquantile(veiling_teff, 0.84, axis=0)
else:
    median_veiling = np.full(70, np.nan)
    lower_percentile_veiling = np.full(70, np.nan)
    upper_percentile_veiling = np.full(70, np.nan)

lower_temps.append(0)
upper_temps.append(2300)
median_veilings.append(median_veiling)
lower_percentile_veilings.append(lower_percentile_veiling)
upper_percentile_veilings.append(upper_percentile_veiling)

for lower_temp in range(2300, 11000, 100):
    upper_temp = lower_temp + 100
    indices = np.where((s['pred_logteff_1'] >= np.log10(lower_temp)) & (s['pred_logteff_1'] < np.log10(upper_temp)))[0]
    if len(indices) > 0:
        veiling_teff = np.array(s['veiling_arr'][indices])
        median_veiling = np.nanquantile(veiling_teff, 0.5, axis=0)
        lower_percentile_veiling = np.nanquantile(veiling_teff, 0.16, axis=0)
        upper_percentile_veiling = np.nanquantile(veiling_teff, 0.84, axis=0)
    else:
        median_veiling = np.full(70, np.nan)
        lower_percentile_veiling = np.full(70, np.nan)
        upper_percentile_veiling = np.full(70, np.nan)

    lower_temps.append(lower_temp)
    upper_temps.append(upper_temp)
    median_veilings.append(median_veiling)
    lower_percentile_veilings.append(lower_percentile_veiling)
    upper_percentile_veilings.append(upper_percentile_veiling)

indices = np.where(s['pred_logteff_1'] >= np.log10(11000))[0]
if len(indices) > 0:
    veiling_teff = np.array(s['veiling_arr'][indices])
    median_veiling = np.nanquantile(veiling_teff, 0.5, axis=0)
    lower_percentile_veiling = np.nanquantile(veiling_teff, 0.16, axis=0)
    upper_percentile_veiling = np.nanquantile(veiling_teff, 0.84, axis=0)
else:
    median_veiling = np.full(70, np.nan)
    lower_percentile_veiling = np.full(70, np.nan)
    upper_percentile_veiling = np.full(70, np.nan)

lower_temps.append(11000)
upper_temps.append(np.inf)
median_veilings.append(median_veiling)
lower_percentile_veilings.append(lower_percentile_veiling)
upper_percentile_veilings.append(upper_percentile_veiling)

t = Table()
t['Lower_Temp'] = Column(lower_temps, unit=u.K, description='Lower bound of temperature range')
t['Upper_Temp'] = Column(upper_temps, unit=u.K, description='Upper bound of temperature range')
t['Median_Veiling'] = median_veilings
t['Lower_Uncertainty'] = np.array(median_veilings) - np.array(lower_percentile_veilings)
t['Upper_Uncertainty'] = np.array(median_veilings) - np.array(upper_percentile_veilings)

arr = np.linspace(3566.9744, 10318.104, 70)
t["Wavelength"] = Column(length=len(t),dtype=float, shape=(70,))+np.nan
t["Wavelength"] = arr

t.write('binned_ctts_veiling.fits', overwrite=True)


/home/serat/miniconda3/envs/stenv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,
